给定一个问题和解决方案，先扩写它们，再生成专利背景介绍、权要和实施方式。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_rewriteq = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据发明人提供的简短描述，重新编写专利发明点要解决问题的说明。
问题说明需要包含以下几个问题涉及的内容：1，待解决的问题属于哪个技术领域？2，在这个领域中，现有软件或者技术存在什么样的问题或者局限性？3，发明点要解决的具体问题是什么？
如果原来的描述中不包含上述问题的答案，请自动补充相关的内容。

待解决问题的简单描述: {raw_question}

根据上述简单描述，重新生成的更为完整和严谨的问题描述是：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [4]:
template_rewritea = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据待解决问题的描述，以及简单的解决方案描述，重新编写专利发明点的解决方案。
解决方案要包含以下几个问题涉及的内容：待解决的问题属于哪个技术领域？在这个领域中，现有软件或者技术存在什么样的问题或者局限性？发明点要解决的具体问题是什么？
解决方案的基本思路是什么？它的功能和特征是什么？它如何解决上述问题？有哪些优点或者创新？

待解决问题的描述: {question}
解决方案的简单描述: {raw_solution}

根据上述信息，重新生成更为完整和严谨的解决方案描述：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [5]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [6]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [7]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [8]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[rewriteq_chain, rewritea_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "solution", "background", "claims", "explanation"],
    verbose=True)

In [9]:
question = '''为了还原用户UI操作流程，需要识别哪些UI操作是相同的，哪些是不同的。比如，提交报销单，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，
其他的比如出差事由、时间范围等等相差非常大，在页面上看起来也很不一样。但是，它们同属于一个业务操作，应该判断相同操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。
仅用图片相似性无法解决这个问题，因为文字内容变化难以检测；仅用文字比较也不行，因为内容相差很大的UI操作可能属于同一个业务操作。
'''

solution = '''使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的。另外，再提取一两个关键标签，代表当前操作的内容。只要窗口区域相似，
关键标签相同，就可以判断两个UI操作属于同一个业务操作。识别窗口区域的方法是canny，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。
提交关键标签，是指使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断窗口的相似性。
'''

result = overall_chain({'raw_question': question, 'raw_solution': solution})



> Entering new SequentialChain chain...

> Finished chain.


In [10]:
print(result['question'])

1. 待解决的问题属于人机交互技术领域。
2. 在这个领域中，现有软件或技术存在的问题或局限性是，对于复杂的业务操作，很难准确识别哪些UI操作是相同的，哪些是不同的。目前常用的图片相似性比较和文字比较方法都难以准确判断不同操作是否属于同一个业务操作，因此无法还原用户UI操作流程。
3. 发明点要解决的具体问题是，提供一种有效的方法来识别哪些UI操作是相同的，哪些是不同的，从而能够准确还原用户UI操作流程。该方法需要综合考虑UI操作的视觉相似性和文字内容相似性，通过智能算法进行分析和比较，能够快速准确地判断不同操作是否属于同一个业务操作，从而提高业务操作的还原度和准确度。


In [11]:
print(result['solution'])

本专利发明点属于人机交互技术领域。在这个领域中，现有软件或技术存在的问题或局限性是，对于复杂的业务操作，很难准确识别哪些UI操作是相同的，哪些是不同的。现有的图片相似性比较和文字比较方法都难以准确判断不同操作是否属于同一个业务操作，因此无法还原用户UI操作流程。

本发明点要解决的具体问题是，提供一种有效的方法来识别哪些UI操作是相同的，哪些是不同的，从而能够准确还原用户UI操作流程。该方法需要综合考虑UI操作的视觉相似性和文字内容相似性，通过智能算法进行分析和比较，能够快速准确地判断不同操作是否属于同一个业务操作，从而提高业务操作的还原度和准确度。

本发明点的基本思路是，首先使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的。另外，再提取一两个关键标签，代表当前操作的内容。只要窗口区域相似，关键标签相同，就可以判断两个UI操作属于同一个业务操作。

识别窗口区域的方法是canny，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。提交关键标签，是指使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断窗口的相似性。

本发明点的功能和特征包括两个方面：一是综合考虑UI操作的视觉相似性和文字内容相似性，能够快速准确地判断不同操作是否属于同一个业务操作；二是使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的，并且再提取一两个关键标签，代表当前操作的内容，从而提高业务操作的还原度和准确度。

本发明点的优点或创新之处包括：一是对于复杂的业务操作，可以准确识别哪些UI操作是相同的，哪些是不同的，从而提高业务操作的还原度和准确度；二是综合考虑UI操作的视觉相似性和文字内容相似性，提高了识别的准确性和可靠性；三是使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的，并且再提取一两个关键标签，代表当前操作的内容，从而提高业务操作的还原度和准确度。


In [12]:
print(result['background'])

本发明点属于人机交互技术领域，目前该领域中存在的问题是，对于复杂的业务操作，很难准确识别哪些UI操作是相同的，哪些是不同的。现有的图片相似性比较和文字比较方法都难以准确判断不同操作是否属于同一个业务操作，从而无法还原用户UI操作流程。为了解决这个问题，本发明点提供了一种有效的方法来识别哪些UI操作是相同的，哪些是不同的，从而能够准确还原用户UI操作流程。该方法综合考虑UI操作的视觉相似性和文字内容相似性，通过智能算法进行分析和比较，能够快速准确地判断不同操作是否属于同一个业务操作，从而提高业务操作的还原度和准确度。本发明点的优点在于综合考虑了UI操作的视觉相似性和文字内容相似性，提高了识别的准确性和可靠性，并且使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的，并且再提取一两个关键标签，代表当前操作的内容，从而提高业务操作的还原度和准确度。


In [13]:
print(result['claims'])

1. 一种用于识别哪些UI操作是相同的，哪些是不同的方法，包括以下步骤：
a. 使用CV算法识别窗口/应用的区域；
b. 提取一两个关键标签，代表当前操作的内容；
c. 综合考虑UI操作的视觉相似性和文字内容相似性，通过智能算法进行分析和比较，判断不同操作是否属于同一个业务操作。

2. 根据权利要求1所述的方法，进一步提高业务操作的还原度和准确度。

3. 根据权利要求1或2所述的方法，在人机交互技术领域中应用的计算机程序产品。


In [14]:
print(result['explanation'])

权利要求1的具体实现步骤：

a. 使用CV算法识别窗口/应用的区域：首先，通过canny算法对图像进行边缘检测，得到窗口/应用的边框信息。然后，通过对边框信息的处理和分析，得到窗口/应用的区域信息。最后，将窗口/应用的区域信息进行存储和管理。

示例说明：用户在一个应用中进行了多个不同的操作，每个操作都会在应用窗口中产生一个新的界面，界面的布局和内容可能不同。通过使用canny算法对图像进行边缘检测，可以得到应用窗口的边框信息。然后，通过对边框信息的处理和分析，得到窗口的区域信息，如窗口的大小和位置等。最后，将窗口的区域信息进行存储和管理，以便后续的操作识别和比较。

b. 提取一两个关键标签，代表当前操作的内容：首先，使用OCR技术对当前窗口中的所有文字进行识别，并将识别结果进行存储。然后，根据用户的操作内容，提取关键位置的文字作为关键标签，并将关键标签进行存储和管理。

示例说明：用户在应用窗口中进行了多个不同的操作，每个操作对应的界面中都包含一些文字信息。使用OCR技术对当前窗口中的所有文字进行识别，并将识别结果进行存储。然后，根据用户的操作内容，提取关键位置的文字作为关键标签，如窗口标题、按钮文字等，以便后续的操作识别和比较。

c. 综合考虑UI操作的视觉相似性和文字内容相似性，通过智能算法进行分析和比较，判断不同操作是否属于同一个业务操作：首先，将待比较的两个操作的窗口区域和关键标签信息进行提取和匹配。然后，通过比较两个操作的窗口区域和关键标签信息的相似度，确定两个操作是否属于同一个业务操作。

示例说明：用户在应用窗口中进行了多个不同的操作，每个操作对应的界面中都包含一些文字信息和UI元素。通过提取窗口区域和关键标签，可以将多个操作进行识别和分类。然后，通过比较两个操作的窗口区域和关键标签信息的相似度，可以确定两个操作是否属于同一个业务操作。例如，用户在同一个应用窗口中点击了两个不同的按钮，通过比较按钮的文字信息和UI元素等，可以确定这两个操作属于同一个业务操作。


In [15]:
with open('patents/patent-output4c.txt', 'w') as f:
    f.write('question: ' + result['raw_question'])    
    f.write('\n\nsolution: ' + result['raw_solution'])
    f.write('\n\nbackground: ' + result['background'])
    f.write('\n\nclaims: ' + result['claims'])
    f.write('\n\nexplanation: ' + result['explanation'])